In [3]:
import torch
import pandas as pd 

import json 
from math import sqrt

In [18]:
Z_99_PERCENT = 2.576

def ci(t): 
    return (
        Z_99_PERCENT * 
        (t.std() / sqrt(t.size(0)))
    ).item()

def to_dict(t):
    t = t.float().sort().values
    quartile = t.size(0) // 4
    trunc = t[quartile:-quartile]
    trunc_mean = trunc.mean().item()
    ci_range = ci(trunc)

    return {
        'mean': t.mean().item(),
        'std': t.std().item(),
        'max': t.max().item(),
        'min': t.min().item(),
        'trunc-mean': trunc_mean,
        'CI-low': trunc_mean - ci_range,
        'CI-high': trunc_mean + ci_range
    }

def get_results(s):
    data = torch.load(f'../results/001lr/{s}_eval.pt')
    lens = data['lens']; rews = data['rews']

    return {'rewards': to_dict(rews), 'episode lens': to_dict(lens)}


In [8]:
import pandas as pd 
def compare(n):
    d_last = get_results(f'ppo_{n}N_0_last')
    d_best = get_results(f'ppo_{n}N_0')

    last_r = d_last['rewards']
    last_r['name'] = 'Last'
    last_l = d_last['episode lens']
    last_l['name'] = 'Last'

    best_r = d_best['rewards']
    best_r['name'] = 'Best'
    best_l = d_best['episode lens']
    best_l['name'] = 'Best'

    df_r = pd.DataFrame([last_r, best_r])
    df_l = pd.DataFrame([last_l, best_l])

    return df_r, df_l 

In [20]:
r10,_ = compare(10)
r20,_ = compare(20)
r40,_ = compare(40)

In [21]:
r10.transpose()

,0,1
mean,230.670929,237.711655
std,120.066246,128.777908
max,549.098022,549.098022
min,153.199768,152.899765
trunc-mean,176.752945,178.781372
CI-low,171.643932,173.410018
CI-high,181.861958,184.152726
name,Last,Best


In [22]:
r20.transpose()

,0,1
mean,185.510773,182.088882
std,93.204048,86.744682
max,574.052246,574.052246
min,135.399719,138.399658
trunc-mean,154.677399,154.743027
CI-low,153.200727,153.387525
CI-high,156.15407,156.098529
name,Last,Best


In [23]:
r40.transpose()

,0,1
mean,157.386642,158.14418
std,5.423238,5.946174
max,183.775558,180.850983
min,143.425644,141.050522
trunc-mean,157.144302,157.758072
CI-low,156.839932,157.375715
CI-high,157.448673,158.140429
name,Last,Best
